<a href="https://colab.research.google.com/github/Nitin-Dwivedi-7/COVID_19-Test-Prediction-Using-CT-scan-Images/blob/main/DenseNet121_major_25GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import Keras packages
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import load_img
import random

import numpy as np
from keras import applications
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.utils import get_file

In [4]:
import pickle 
file = "/content/drive/My Drive/data_5K.pkl"
f_o=open(file,"rb")
data=pickle.load(f_o)
f_o.close()
file= "/content/drive/My Drive/labels_5k.pkl"
f_o=open(file,"rb")
labels=pickle.load(f_o)
f_o.close()

In [5]:
data = np.array(data)
labels = np.array(labels)

In [6]:
img_width, img_height = 256, 256        # Resolution of inputs
#train_data_dir = "train"           # Folder of train samples
#validation_data_dir = "val" # Folder of validation samples44
#nb_train_samples = 10000                # Number of train samples
#nb_validation_samples = 9500            # Number of validation samples
batch_size = 64                        # Batch size
epochs = 20                # Maximum number of epochs

In [7]:
# Load ResNet50
model=keras.applications.DenseNet121(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))


29089792/29084464 [==============================] - 1s 0us/step


In [8]:
# Freeze first 15 layers
for layer in model.layers[:45]:
	layer.trainable = False
for layer in model.layers[45:]:
   layer.trainable = True

In [9]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x) # 4-way softmax classifier at the end


In [10]:
nb_train_samples = 2543                # Number of train samples
nb_validation_samples = 2261            # Number of validation samples

# Initializing the CNN


In [11]:
from keras.utils.np_utils import to_categorical   

categorical_labels = to_categorical(labels, num_classes=2)


from sklearn.model_selection import train_test_split

#X_train1, X_test0, ytrain1, ytest0 = train_test_split(data, categorical_labels, test_size=0.1,
                                                    #random_state=random.randint(0,100))
X_train1 = data
ytrain1 = categorical_labels

for index in range(10):
  classifier = Model(model.input,predictions)
  classifier.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-3, momentum=0.9), metrics=["accuracy"])


X_train, X_test1, ytrain, ytest1 = train_test_split(X_train1, ytrain1, test_size=0.1,
                                                    random_state=random.randint(0,100))
X_val, X_test, yval, ytest = train_test_split(X_test1, ytest1, test_size=0.5,
                                                    random_state=random.randint(0,100))
    


In [12]:
from keras.preprocessing.image import ImageDataGenerator
    
train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    
test_datagen = ImageDataGenerator(rescale=1./255)
    
val_datagen = ImageDataGenerator(rescale=1./255)
    
training_set = train_datagen.flow(
            X_train, ytrain,
            batch_size=64)
    
val_set = val_datagen.flow(
            X_val, yval,
            batch_size=64)
    
test_set = test_datagen.flow(
            X_test, ytest,
            batch_size=64)
    
X, y = test_set.next()
    
classifier.fit_generator(
            training_set,
            steps_per_epoch=20,
            epochs=20,
            validation_data=val_set,
            validation_steps=100)
    
    #w_file = 'Wilson_vgg16_model_weights_k10.h5'
    #classifier.save_weights(w_file)
    
    
    

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
20/20 [==============================] - 17s 848ms/step - loss: 0.5656 - accuracy: 0.8359 - val_loss: 1.1479 - val_accuracy: 0.7062
Epoch 2/20
20/20 [==============================] - 14s 702ms/step - loss: 0.1686 - accuracy: 0.9578
Epoch 3/20
20/20 [==============================] - 15s 753ms/step - loss: 0.0850 - accuracy: 0.9787
Epoch 4/20
20/20 [==============================] - 14s 714ms/step - loss: 0.0317 - accuracy: 0.9898
Epoch 5/20
20/20 [==============================] - 14s 707ms/step - loss: 0.0182 - accuracy: 0.9930
Epoch 6/20
20/20 [==============================] - 14s 710ms/step - loss: 0.0195 - accuracy: 0.9922
Epoch 7/20
20/20 [==============================] - 14s 711ms/step - loss: 0.0188 - accuracy: 0.9953
Epoch 8/20
20/20 [==============================] - 14s 700ms/step - loss: 0.0132 - accuracy: 0.9961
Epoch 9/20
20/20 [==============================] - 14s 702ms/step - loss:

In [13]:
arr = classifier.evaluate(X,y)

2/2 [==============================] - 0s 38ms/step - loss: 0.0510 - accuracy: 0.9844


In [14]:
print(arr)

[0.05104001238942146, 0.984375]


In [15]:


arr = classifier.predict(X)
arr = np.argmax(arr, axis=1)

print(arr)
print(y)


[1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 0 0
 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
